<a href="https://colab.research.google.com/github/Suhail372/files_for_chatbot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Suhail372/files_for_chatbot

Cloning into 'files_for_chatbot'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 191 (delta 129), reused 160 (delta 109), pack-reused 0 (from 0)
Receiving objects: 100% (191/191), 12.37 MiB | 25.18 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [ ]:
import os
os.chdir("/content/files_for_chatbot")

In [ ]:
import json
import random
with open("F:\Chat_Proj\\files_for_chatbot\\testcases\\two variable testcases.json","r") as file:
    test_cases=json.load(file)

with open("combined files\\cleaned_and_combined_hyd.json","r") as file:
    data=json.load(file)

for i in random.sample(test_cases[200:300],5):
    query=i["query"]
    ans=i["ans"]
    count=0
    results=[]
    target=min(3,len(ans))
    for j in data:
        if j["Id"] in ans:
            count+=1
            results.append(j['text data'])
            if count==target:
                break
    print(f"""Query: {query}
Search results:{results}""")

Query: CBSE board schools with Handball
Search results:['Name: Indus Universal School~Category: International Schools~Location: Near Tulsi Gardens, JJ Nagar Colony(PO), Yapral, Sainikpuri, Shivaji Nagar, Sainikpuri, Secunderabad, Telangana 500087, India~Faculty: ~Sports: Athletics, Badminton, Basketball, Cricket, Football, Gymnastics, Handball, Hockey, Kabaddi, Kho-Kho, Swimming, Table-tennis, Tennis, Volleyball~Amenities: Transport, Medical Facility, Laboratory, Smart Classrooms, Computers Facility, Library~Board: CBSE~Years: -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12~Fee: -1~Since: 2006~Strength: Not Available~', 'Name: Sanskriti The School~Category: Public Schools~Location: Kothapet Fruit Market\r\nSaroornagar Road, Dilsukhnagar\r\nHyderabad - 500035~Faculty: Mrs. S. Swati Principal / M.A., M.Ed Mr. Vikranth Terala Director / M.B.A.~Sports: Carroms, Chess, Cricket, Football, Handball~Amenities: Transport, Medical Facility, Laboratory, Smart Classrooms, Computers Facility, Library~

In [ ]:
import json
import os
os.chdir("/content/files_for_chatbot")

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def load_llama_model():
    '''bnb_config = BitsAndBytesConfig(load_in_8bit=True,
                                    bnb_8bit_use_double_quant=True,
                                    bnb_8bit_quant_type="nf4",
                                    bnb_8bit_compute_dtype=torch.bfloat16)'''
    llama_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
    llama_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct").to(device)
    llama_model.temperature = 1.0
    llama_model.top_p = 0.95
    return llama_tokenizer, llama_model



def generate_query(llama_tokenizer, llama_model, query, history):
    input_prompt = f"""
    <>
        Generate a question similar to previous queries, and given user query, make it concise and do not generate an answer to the query.
    <>
    [INST]
    History: {history}
    Query: {query}
    [/INST]
    """
    inputs = llama_tokenizer(input_prompt, return_tensors="pt", padding=True).to("cuda")
    outputs = llama_model.generate(**inputs, max_new_tokens=1000)
    result_prompt = llama_tokenizer.batch_decode(outputs)[0]
    return result_prompt.replace(input_prompt, "").replace("","").replace("","")

def not_query_response(llama_tokenizer, llama_model, query):
    query = query.lower()
    input_prompt = f"""
        <>
            You are a greeting checker and check if the sentence is explicitly greeting or not.
            Explain if it is a greeting or not. If it is NOT a greeting, i mean not a greeting then and then only please must include number 10393 in response.
        <>
        [INST]
        Sentence: {query}
        [/INST]
        """
    inputs = llama_tokenizer(input_prompt, return_tensors="pt", padding=True).to("cuda")
    outputs = llama_model.generate(**inputs, max_new_tokens=1000)
    result_prompt = llama_tokenizer.batch_decode(outputs)[0]
    result_prompt = result_prompt.replace(input_prompt, "")
    if "10393" in result_prompt:
        return "Please enter proper query", result_prompt
    greeting_prompt = f"""
        <>
            Please respond to the greeting appropriately and concisely
        <>
        [INST]
        Greeting: {query}
        [/INST]
        """
    inputs = llama_tokenizer(greeting_prompt, return_tensors="pt", padding=True).to("cuda")
    outputs = llama_model.generate(**inputs, max_new_tokens=1000)
    result = llama_tokenizer.batch_decode(outputs)[0]
    result = result.replace(greeting_prompt, "")
    return result

def is_query(query):
    query = query.lower()
    my_list = ["school", "schools", "facilities", "amenities", "sports", "faculty", "fees", "institute", "organisation", "org", "inst", "scl", "schol"]
    if any(item in query for item in my_list):
        return True
    return False

def requires_context(query):
    query = query.lower()
    context_keywords = ["previous", "last", "before", "history", "context", "same"]
    if any(item in query for item in context_keywords):
        return True
    return False


In [27]:
def generate_chat_response(llama_tokenizer, llama_model, query, data):


    input_prompt = f"""
    <>You are a chatbot assistant that recommends schools to the user by describing the school's information. Generate the concise answer for to the query given using the search results given and nothing else in response field.
    <>
    [INST]
    User Query: {query}
    Search results: {data}
    [/INST]
    """

    inputs = llama_tokenizer([input_prompt], return_tensors="pt").to("cuda")
    outputs = llama_model.generate(**inputs, max_new_tokens=1000, pad_token_id=llama_model.config.eos_token_id)
    result_prompt = llama_tokenizer.decode(outputs[0])
    result_prompt = result_prompt.replace(input_prompt, "")
    return result_prompt.replace("","").replace("","")

In [22]:
llama_tokenizer, llama_model = load_llama_model()

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [10]:
query="Query: IGCSE board schools with Ice-Hockey"
data="['Name: Oakridge International School~Category: International Schools~Location: Bowrampet, Near Bachupally, Hyderabad  500 043, Telangana, India~Faculty: Ms. Baljeet Oberoi Principal / Masters in Mathematics~Sports: Athletics, Badminton, Basketball, Carroms, Chess, Cricket, Football, Gymnastics, Hockey, Ice-Hockey, Karate, Lawn Tennis, Skating, Swimming, Tennis, Throwball, Volleyball, Yoga~Amenities: Transport, Medical Facility, Laboratory, Smart Classrooms, Computers Facility, Library~Board: CBSE, International Baccalaureate, Cambridge IGCSE~Years: -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12~Fee: 21800000, 71700000~Since: 2001~Strength: Not Available~']"

In [26]:
generate_chat_response(llama_tokenizer=llama_tokenizer, llama_model=llama_model, query=query, data=data)

' IGCSE board schools with ice-hockey include Oakridge International School located in Bowrampet, near Bachupally, Hyderabad. The school has facilities like Athletics, Badminton, Basketball, Carroms, Chess, Cricket, Football, Gymnastics, Hockey, Ice-Hockey, Karate, Lawn Tennis, Skating, Swimming, Tennis, Throwball, Volleyball, and Yoga. It also offers transport, medical facility, laboratory, smart classrooms, computers facility, library, and an international curriculum including CBSE, IB, and Cambridge IGCSE. The school fee is around INR 21.8 Lakhs per annum. \n    This school provides quality education to its students while offering various sports activities, making it a suitable choice for students who enjoy sports as well as academic excellence.<|endoftext|>'